# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [1]:
from typing import Tuple, List

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove
from copy import copy
import numpy as np
from math import inf



Wielkość planszy

In [2]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [3]:
class Max(Player):
    def __init__(self, char: str) -> None:
        super().__init__(char)

class Min(Player):
    def __init__(self, char: str) -> None:
        super().__init__(char)

In [99]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        if type(game.first_player) is not Max and type(game.first_player) is not Min:
            raise ValueError("The first player is not Max nor Min")
        if type(game.second_player) is not Max and type(game.second_player) is not Min:
            raise ValueError("The second player is not Max nor Min")
        if type(game.first_player) is type(game.second_player):
            raise ValueError("Both players are the same type; One should be Max and one should be Min")
        self.game = game

    def evaluate_position(self, player: Player)->float:
        if self.game.is_finished():
            winner = self.game.get_winner()
            if type(winner) is Max:
                return inf
            elif type(winner) is Min:
                return -inf
            else:
                return 0.0
        else:
            evaluation = np.random.normal()
            return evaluation

    def _evaluate_score(self, player) -> float:
        score = 0
        middle_column = len(self.game.state.fields)//2
        chars_in_center = self.game.state.fields[middle_column].count(player.char)
        score += chars_in_center * 5
        return score
        
        
    def _evaluate_next_state(self, move: int, depth: int, alpha:float, beta: float, is_maximizing_player: bool) -> float:
        evaluated_game = copy(self.game)
        evaluated_game.make_move(ConnectFourMove(move))
        new_solver = MinMaxSolver(evaluated_game)
        return new_solver.minimax(depth-1, alpha, beta, not is_maximizing_player)[1]

    def is_valid_move(self, col_index:int)->bool:
        return self.game.state.fields[col_index][ROW_COUNT - 1] is None
    
    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""
        valid_moves = self.get_valid_moves()
        if self.game.is_finished() or depth == 0:
            players = self.game.get_players()
            max_player = players[0] if type(players[0]) is Max else players[1]
            return None, self.evaluate_position(max_player)
        else:
            best_move = valid_moves[np.random.choice(len(valid_moves))]
            if is_maximizing_player:
                evaluation = -inf
                for move in valid_moves:
                    new_evaluation = self._evaluate_next_state(move, depth, alpha, beta, is_maximizing_player)
                    if new_evaluation > evaluation:
                        evaluation = new_evaluation
                        best_move = move
                    alpha = max(alpha, evaluation)
                    if alpha >= beta:
                        break
                return best_move, alpha
            else:
                evaluation = inf
                for move in valid_moves:
                    new_evaluation = self._evaluate_next_state(move, depth, alpha, beta, is_maximizing_player)
                    if new_evaluation < evaluation:
                        evaluation = new_evaluation
                        best_move = move
                    beta = min(beta, evaluation)
                    if alpha >= beta:
                        break
                return best_move, beta

            
    def get_valid_moves(self) -> list:
        valid_moves = []
        for column in range(COLUMN_COUNT):
            if self.is_valid_move(column):
                valid_moves.append(column)
        return valid_moves

Rozgrywka

In [97]:
p1 = Max("a")
p2 = Min("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
solver = MinMaxSolver(game)
while not game.is_finished():
    move, eval = solver.minimax(5, -1000.1, 1000.1, type(solver.game.get_current_player()) is Max)
    game.make_move(ConnectFourMove(move))
    
print(game.state)



Current player: b
[b][b][a][ ][ ][ ][ ]
[a][a][a][ ][ ][b][ ]
[b][b][b][ ][ ][b][b]
[a][a][a][a][a][a][a]
[b][a][a][b][b][a][b]
[b][b][a][a][b][a][b]


a


In [38]:
print(game.state)
move, eval = solver.minimax(2, -1000.1, 1000.1, type(solver.game.get_current_player()) is Max)
print(game.state.get_winner())

Current player: b
[b][a][a][ ][a][b][a]
[b][b][b][ ][b][a][a]
[a][a][a][ ][a][b][a]
[b][b][b][a][b][b][b]
[b][b][a][b][a][b][a]
[a][a][a][b][a][a][b]


In [101]:

p1 = Max("x")
p2 = Min("o")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
game.make_move(ConnectFourMove(6))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(6))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(6))
game.make_move(ConnectFourMove(3))

s = MinMaxSolver(game)
print(s.minimax(3, -inf, inf, True))



(6, inf)


In [109]:
g1 = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
solver = MinMaxSolver(g1)
while not g1.state.is_finished():
    print(g1.state)
    move = int(input())
    if solver.is_valid_move(move):
        g1.make_move(ConnectFourMove(move))
    else:
        continue
    if g1.get_winner():
        break
    move, eval = solver.minimax(8, -inf, inf, type(solver.game.get_current_player()) is Max)
    print(f"CPU Move: {move}")
    print(f"Eval: {eval}")
    g1.make_move(ConnectFourMove(move))
   

print(f"Winner: {g1.get_winner().char}")
print(g1.state)
    


Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
CPU Move: 5
Eval: 0.9476454731762441
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][x][ ][o][ ]
CPU Move: 3
Eval: 0.9189130004224318
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][x][ ][o][ ]
CPU Move: 1
Eval: 0.8472160438075901
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][o][ ][x][ ][o][ ]
CPU Move: 5
Eval: 0.7611751761173694
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][o][ ]
[ ][o][ ][x][ ][o][ ]
CPU Move: 1
Eval: 0.6025175893580971
Current player: x
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ]